# Pet Insurance Profitability Analysis: A Ratemaking and Reserving Study

## 1. Business Problem
A hypothetical insurer, "Pawtect Insurance," wants to launch a new pet insurance product. The company needs to establish a profitable pricing structure and estimate the required reserves for future claims. This project will develop a comprehensive actuarial analysis to address these needs.

## 2. Project Setup and Data Generation
First, we'll import the necessary libraries and generate a synthetic dataset of 10,000 pet insurance policies. This data will serve as the basis for our analysis.

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Data Generation ---
np.random.seed(42)
num_policies = 10000

data = {
    'policy_id': range(1, num_policies + 1),
    'pet_age': np.random.randint(1, 15, size=num_policies),
    'pet_breed': np.random.choice(['Labrador', 'Poodle', 'Mixed', 'Bulldog', 'Beagle'], size=num_policies),
    'location': np.random.choice(['Urban', 'Rural', 'Suburban'], size=num_policies),
    'coverage_level': np.random.choice(['Basic', 'Standard', 'Premium'], size=num_policies),
    'policy_year': np.random.randint(2020, 2024, size=num_policies)
}

df = pd.DataFrame(data)

def generate_claims(row):
    base_freq = 0.1
    base_sev = 500
    age_factor = 1 + (row['pet_age'] * 0.05)
    breed_factor = {'Labrador': 1.2, 'Poodle': 0.9, 'Mixed': 1.0, 'Bulldog': 1.5, 'Beagle': 0.8}.get(row['pet_breed'], 1)
    location_factor = {'Urban': 1.3, 'Rural': 0.8, 'Suburban': 1.1}.get(row['location'], 1)
    claim_freq = base_freq * age_factor * breed_factor * location_factor
    claim_sev = base_sev * age_factor * breed_factor * location_factor
    num_claims = np.random.poisson(claim_freq)
    claims_incurred = sum(np.random.gamma(2, scale=claim_sev/2, size=num_claims))
    return claims_incurred

df['claims_incurred'] = df.apply(generate_claims, axis=1)
df['annual_premium'] = 200 + (df['pet_age'] * 20) + df['claims_incurred'].apply(lambda x: max(0, x * np.random.uniform(0.3, 0.6)))
df['claim_status'] = df['claims_incurred'].apply(lambda x: 'Paid' if x > 0 and np.random.rand() > 0.3 else 'Outstanding')

print("--- Synthetic Data Generated ---")
df.head()

## 3. Ratemaking Analysis
In this section, we'll perform an exploratory data analysis (EDA) to understand the key drivers of claims. We will then build a Generalized Linear Model (GLM) to create a risk-based pricing structure.

In [ ]:
# Exploratory Data Analysis (EDA)
plt.figure(figsize=(12, 6))
sns.histplot(df['claims_incurred'], bins=50, kde=True)
plt.title('Distribution of Claims Incurred')
plt.xlabel('Claims Incurred')
plt.ylabel('Frequency')
plt.show()

# Loss ratio by pet breed
loss_ratio_breed = df.groupby('pet_breed').agg({'claims_incurred': 'sum', 'annual_premium': 'sum'})
loss_ratio_breed['loss_ratio'] = loss_ratio_breed['claims_incurred'] / loss_ratio_breed['annual_premium']
print("Loss Ratio by Pet Breed:")
print(loss_ratio_breed)

In [ ]:

# Loss ratio by location
loss_ratio_location = df.groupby('location').agg({'claims_incurred': 'sum', 'annual_premium': 'sum'})
loss_ratio_location['loss_ratio'] = loss_ratio_location['claims_incurred'] / loss_ratio_location['annual_premium']
print("\nLoss Ratio by Location:")
print(loss_ratio_location)

# Visualize loss ratios
fig, ax = plt.subplots(1, 2, figsize=(18, 6))
loss_ratio_breed.plot(kind='bar', y='loss_ratio', ax=ax[0], title='Loss Ratio by Pet Breed')
ax[0].set_ylabel('Loss Ratio')
loss_ratio_location.plot(kind='bar', y='loss_ratio', ax=ax[1], title='Loss Ratio by Location')
ax[1].set_ylabel('Loss Ratio')
plt.show()


In [ ]:

# Relationship between Pet Age and Claims Incurred
plt.figure(figsize=(12, 6))
sns.scatterplot(data=df, x='pet_age', y='claims_incurred', alpha=0.5)
plt.title('Claims Incurred vs. Pet Age')
plt.xlabel('Pet Age (Years)')
plt.ylabel('Claims Incurred ($)')
plt.show()


In [ ]:
# GLM for Pricing
df_glm = pd.get_dummies(df, columns=['pet_breed', 'location', 'coverage_level'], drop_first=True, dtype=int)
df_glm['claim_count'] = df['claims_incurred'].apply(lambda x: 1 if x > 0 else 0)

# Define features for the model
features = ['pet_age', 'pet_breed_Labrador', 'pet_breed_Mixed', 'pet_breed_Poodle', 'location_Suburban', 'location_Urban']
X = sm.add_constant(df_glm[features])

# Frequency model (Poisson GLM)
freq_model = sm.GLM(df_glm['claim_count'], X, family=sm.families.Poisson()).fit()
print("--- GLM Frequency Model Results ---")
print(freq_model.summary())

# Severity model (Gamma GLM)
df_sev = df[df['claims_incurred'] > 0]
df_glm_sev = df_glm.loc[df_sev.index]
X_sev = sm.add_constant(df_glm_sev[features])

sev_model = sm.GLM(df_sev['claims_incurred'], X_sev, family=sm.families.Gamma(link=sm.families.links.Log())).fit()
print("--- GLM Severity Model Results ---")
print(sev_model.summary())

## 4. Reserving Analysis
Here, we'll use the chain-ladder method to estimate the ultimate losses and calculate the Incurred But Not Reported (IBNR) reserves. This is a crucial step in ensuring the financial health of the insurer.

In [ ]:
# Create a sample claims dataset for reserving
claims_data = {
    'accident_year': np.random.randint(2018, 2022, 1000),
    'development_lag': np.random.randint(1, 5, 1000),
    'paid_losses': np.random.randint(100, 5000, 1000)
}
claims_df = pd.DataFrame(claims_data)
claims_df['paid_losses'] = claims_df.groupby(['accident_year', 'development_lag'])['paid_losses'].transform('sum')
claims_df = claims_df.drop_duplicates()

# Create loss triangle
loss_triangle = claims_df.pivot(index='accident_year', columns='development_lag', values='paid_losses').fillna(0).astype(float)
print("--- Loss Development Triangle ---")
print(loss_triangle)

# Chain-Ladder Method
def chain_ladder(triangle):
    triangle = triangle.copy()
    for i in range(triangle.shape[1] - 1):
        col1 = triangle.iloc[:, i]
        col2 = triangle.iloc[:, i+1]
        factor = col2.sum() / col1.sum()
        for j in range(triangle.shape[0] - (i + 1)):
            triangle.iloc[j, i+1] = triangle.iloc[j, i] * factor
    return triangle

projected_triangle = chain_ladder(loss_triangle)
print("--- Projected Loss Triangle (Chain-Ladder) ---")
print(projected_triangle)

# Calculate ultimate losses and IBNR
ultimate_losses = projected_triangle.iloc[:, -1]
paid_to_date = loss_triangle.max(axis=1)
ibnr = ultimate_losses - paid_to_date

reserves = pd.DataFrame({'Ultimate Losses': ultimate_losses, 'Paid to Date': paid_to_date, 'IBNR': ibnr})
print("\n--- Reserve Estimation ---")
print(reserves)
print(f"\nTotal IBNR Reserves: ${reserves['IBNR'].sum():,.2f}")

In [ ]:

# Visualize IBNR Reserves
plt.figure(figsize=(10, 6))
reserves.plot(kind='bar', y=['Paid to Date', 'IBNR'], stacked=True, figsize=(10, 7))
plt.title('Reserve Estimation (Paid-to-Date vs. IBNR)')
plt.xlabel('Accident Year')
plt.ylabel('Amount ($)')
plt.show()


## 5. Conclusion and Next Steps
This analysis provides a comprehensive actuarial framework for Pawtect Insurance's new pet insurance product. The ratemaking model offers a data-driven approach to pricing, while the reserving analysis ensures the company's solvency.
The next steps include:
1. Implementing the ratemaking model in the company's system.
2. Conducting ongoing monitoring of claims experience and adjusting rates accordingly.
3. Exploring additional factors that may influence claim costs (e.g., breed-specific risks).
4. Developing strategies to manage potential adverse development in reserves.
By following these recommendations, Pawtect can effectively price its pet insurance product and maintain financial stability over time.


## 6. Proposed Premium Structure and Performance Monitoring
Based on the GLM results, we can construct a new risk-based premium for each policy. The pure premium is calculated as the product of the predicted claim frequency and the predicted claim severity. We will then add a loading for expenses and profit to arrive at the final proposed premium.


In [ ]:
# Predict frequency and severity for each policy
df_glm['predicted_freq'] = freq_model.predict(X)
df_glm['predicted_sev'] = sev_model.predict(X)

# Calculate pure premium
df_glm['pure_premium'] = df_glm['predicted_freq'] * df_glm['predicted_sev']

# Add a loading for expenses and profit margin. A 68% loading targets a loss ratio of 32%.
loading_factor = 0.68
df_glm['proposed_premium'] = df_glm['pure_premium'] / (1 - loading_factor)

# Compare old vs. proposed premium
print("--- Premium Comparison ---")
print(df[['annual_premium']].describe())
print(df_glm[['proposed_premium']].describe())

# Plot the distribution of proposed premiums
plt.figure(figsize=(12, 6))
sns.histplot(df_glm['proposed_premium'], bins=50, kde=True, color='green')
plt.title('Distribution of Proposed Premiums')
plt.xlabel('Proposed Premium')
plt.ylabel('Frequency')
plt.show()

In [ ]:

# Compare the distribution of old vs. proposed premiums
plt.figure(figsize=(14, 7))
sns.histplot(df['annual_premium'], bins=50, kde=True, color='blue', label='Original Premium')
sns.histplot(df_glm['proposed_premium'], bins=50, kde=True, color='green', label='Proposed Premium')
plt.title('Distribution of Original vs. Proposed Premiums')
plt.xlabel('Annual Premium')
plt.ylabel('Frequency')
plt.legend()
plt.show()


In [ ]:

# --- Profitability Analysis ---
total_claims_incurred = df['claims_incurred'].sum()
total_original_premium = df['annual_premium'].sum()
total_proposed_premium = df_glm['proposed_premium'].sum()

original_loss_ratio = total_claims_incurred / total_original_premium
proposed_loss_ratio = total_claims_incurred / total_proposed_premium

print("--- Profitability Comparison ---")
print(f"Original Total Premium: ${total_original_premium:,.2f}")
print(f"Proposed Total Premium: ${total_proposed_premium:,.2f}")
print(f"Original Loss Ratio: {original_loss_ratio:.2%}")
print(f"Proposed Loss Ratio: {proposed_loss_ratio:.2%}")

improvement = (original_loss_ratio - proposed_loss_ratio) / original_loss_ratio
print(f"\nProjected Improvement in Loss Ratio: {improvement:.2%}")



## 7. Incorporating External Data (Veterinary Cost Inflation)
To enhance the accuracy of our models, we can incorporate external data, such as a veterinary cost inflation index. This will allow us to trend the historical claims data to a common cost level, leading to more accurate premium calculations.


In [ ]:
# --- Incorporate Veterinary Cost Inflation ---
vet_inflation = {2020: 0.05, 2021: 0.06, 2022: 0.07, 2023: 0.08}
current_year = 2024

df['inflation_factor'] = df['policy_year'].apply(lambda year: (1 + vet_inflation.get(year, 0))**(current_year - year))
df['claims_incurred_trended'] = df['claims_incurred'] * df['inflation_factor']

# --- Re-run GLM with Trended Data ---
df_glm['claims_incurred_trended'] = df['claims_incurred_trended']
sev_model_trended = sm.GLM(df_glm.loc[df_sev.index, 'claims_incurred_trended'], 
                             X_sev, 
                             family=sm.families.Gamma(link=sm.families.links.Log())).fit()

print("--- GLM Severity Model Results (with Trended Data) ---")
print(sev_model_trended.summary())


## 8. Advanced Reserving Model (Claim-Level Data)
To further refine our reserve estimates, we can use claim-level data. This allows for a more granular analysis of claim development patterns. We'll simulate a claim-level dataset and then use it to build a more sophisticated reserving model.


In [ ]:

# --- Simulate Claim-Level Data for Advanced Reserving ---
num_claims = 5000
claim_level_data = {
    'claim_id': range(1, num_claims + 1),
    'accident_year': np.random.randint(2018, 2022, size=num_claims),
    'report_lag': np.random.geometric(p=0.5, size=num_claims),
    'payment_lag': np.random.geometric(p=0.3, size=num_claims),
    'claim_amount': np.random.lognormal(mean=6, sigma=1.5, size=num_claims)
}
claim_level_df = pd.DataFrame(claim_level_data)
claim_level_df['report_year'] = claim_level_df['accident_year'] + claim_level_df['report_lag']
claim_level_df['payment_year'] = claim_level_df['report_year'] + claim_level_df['payment_lag']

# --- Basic Aggregation for Chain-Ladder (as a comparison) ---
# This is a simplified version of what would be a more complex model
paid_by_ay_dy = claim_level_df.groupby(['accident_year', 'payment_lag'])['claim_amount'].sum().unstack().fillna(0)
print("--- Claim-Level Paid Loss Triangle ---")
print(paid_by_ay_dy.head())

# --- Placeholder for a more advanced model ---
# A true claim-level model would involve more complex methods like individual claim loss reserving models (e.g., GLMs or machine learning models on individual claim features).
# For now, we'll just show the data structure.
print("\n--- Advanced Reserving Model (Placeholder) ---")
print("A more sophisticated model would use individual claim features to project ultimate losses.")
print(claim_level_df.head())
